# Noise Analysis and Deboost

## Importing Modules

In [6]:
import os 
import wget
from threadpoolctl import threadpool_limits
import healpy as hp
import numpy as np
from tqdm import tqdm
from astropy.io import fits as pyfits
import random
import pymp
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
import importlib
import sys
import multiprocessing as mp
from math import sqrt
from math import radians
import gc
import subprocess
gc.disable()
#######################################################################################################
from cmbaberdopp_beta0p8 import * # cmbboost its a module created w/ the functions needed for this work          #
                       # remember to change the path of maps on dipole_doppler_load_low_res function  #
#######################################################################################################
def c_nthreads_limit(n): # Function to change number of threads used on imported C/C++ functions
    threadpool_limits(limits=n, user_api='blas')
    threadpool_limits(limits=n, user_api='openmp')

In [ ]:
nsims = 300

# Download Planck Noise Simulations
for i in tqdm(range(nsims),desc='Downloading noise simulations:'):
    wget.download('http://pla.esac.esa.int/pla/aio/product-action?SIMULATED_MAP.FILE_ID=dx12_v3_smica_noise_mc_'+str(i).zfill(5)+'_raw.fits')
    wget.download('http://pla.esac.esa.int/pla/aio/product-action?SIMULATED_MAP.FILE_ID=dx12_v3_nilc_noise_mc_'+str(i).zfill(5)+'_raw.fits')

In [2]:
# Download Planck final SMICA and NILC maps to get realistic beam function

wget.download('http://pla.esac.esa.int/pla/aio/product-action?MAP.MAP_ID=COM_CMB_IQU-smica_2048_R3.00_full.fits')
wget.download('http://pla.esac.esa.int/pla/aio/product-action?MAP.MAP_ID=COM_CMB_IQU-nilc_2048_R3.00_full.fits')

'COM_CMB_IQU-nilc_2048_R3.00_full.fits'

In [5]:
lmax_var=2048

nilcgb = pyfits.open('COM_CMB_IQU-nilc_2048_R3.00_full.fits')[2].data['INT_BEAM'][0:lmax_var+1]
nilcgb[0]=1
nilcgb[1]=1
nilcgb_P = pyfits.open('COM_CMB_IQU-nilc_2048_R3.00_full.fits')[2].data['POL_BEAM'][0:lmax_var+1]
nilcgb_P[0]=1
nilcgb_P[1]=1

smicagb = pyfits.open('COM_CMB_IQU-smica_2048_R3.00_full.fits')[2].data['INT_BEAM'][0:lmax_var+1]
smicagb[0]=1
smicagb[1]=1
smicagb_P = pyfits.open('COM_CMB_IQU-smica_2048_R3.00_full.fits')[2].data['POL_BEAM'][0:lmax_var+1]
smicagb_P[0]=1
smicagb_P[1]=1

hp.write_cl('smicagb_2018.fits',smicagb)
hp.write_cl('smicagb_P_2018.fits',smicagb_P)
hp.write_cl('nilcgb_2018.fits',nilcgb)
hp.write_cl('nilcgb_P_2018.fits',nilcgb_P)

## NOISE DEBEAMING AND HARMONIC EXPANSION (I,Q,U $\rightarrow$ T,E,B)

In [ ]:
nsim = 300
lmax_var = 2048

masksymm = hp.read_map('temp_symmmask_pr3.fits',verbose=False)

c_nthreads_limit(8)

#not masked
for i in tqdm(range(nsim)):
    nilc_map_I, nilc_map_Q, nilc_map_U = hp.read_map('dx12_v3_nilc_noise_mc_'+str(i).zfill(5)+'_raw.fits',field=(0,1,2))
    alm_T_noise, alm_E_noise, alm_B_noise = hp.map2alm((nilc_map_I, nilc_map_Q, nilc_map_U),lmax=lmax_var,iter=3)
    alm_T_debeamed = hp.almxfl(alm_T_noise,1/(nilcgb[0:lmax_var]))
    alm_E_debeamed = hp.almxfl(alm_E_noise,1/(nilcgb_P[0:lmax_var]))
    alm_B_debeamed = hp.almxfl(alm_B_noise,1/(nilcgb_P[0:lmax_var]))
    hp.write_alm('debeamed_nilc_T_noise_'+str(i)+'.fits',alm_T_debeamed,overwrite=True)
    hp.write_alm('debeamed_nilc_E_noise_'+str(i)+'.fits',alm_E_debeamed,overwrite=True)
    hp.write_alm('debeamed_nilc_B_noise_'+str(i)+'.fits',alm_B_debeamed,overwrite=True)
    
for i in tqdm(range(nsim)):
    smica_map_I, smica_map_Q, smica_map_U = hp.read_map('dx12_v3_smica_noise_mc_'+str(i).zfill(5)+'_raw.fits',field=(0,1,2))
    alm_T_noise, alm_E_noise, alm_B_noise = hp.map2alm((smica_map_I, smica_map_Q, smica_map_U),lmax=lmax_var,iter=3)
    alm_T_debeamed = hp.almxfl(alm_T_noise,1/(smicagb[0:lmax_var]))
    alm_E_debeamed = hp.almxfl(alm_E_noise,1/(smicagb_P[0:lmax_var]))
    alm_B_debeamed = hp.almxfl(alm_B_noise,1/(smicagb_P[0:lmax_var]))
    hp.write_alm('debeamed_smica_T_noise_'+str(i)+'.fits',alm_T_debeamed,overwrite=True)
    hp.write_alm('debeamed_smica_E_noise_'+str(i)+'.fits',alm_E_debeamed,overwrite=True)
    hp.write_alm('debeamed_smica_B_noise_'+str(i)+'.fits',alm_B_debeamed,overwrite=True)

#masked
for i in tqdm(range(nsim)):
    alm_T = hp.read_alm('debeamed_nilc_T_noise_'+str(i)+'.fits')
    alm_E = hp.read_alm('debeamed_nilc_E_noise_'+str(i)+'.fits')
    masked_map_T = hp.alm2map(alm_T,nside=2048)*masksymm
    masked_map_E = hp.alm2map(alm_E,nside=2048)*masksymm
    alm_masked_T = hp.map2alm(masked_map_T,lmax=2048)
    alm_masked_E = hp.map2alm(masked_map_E,lmax=2048)
    hp.write_alm('nilc_symmmasked_T_noise_alm_'+str(i)+'.fits',alm_masked_T)
    hp.write_alm('nilc_symmmasked_E_noise_alm_'+str(i)+'.fits',alm_masked_E)
    
for i in tqdm(range(nsim)):
    alm_T = hp.read_alm('debeamed_smica_T_noise_'+str(i)+'.fits')
    alm_E = hp.read_alm('debeamed_smica_E_noise_'+str(i)+'.fits')
    masked_map_T = hp.alm2map(alm_T,nside=2048)*masksymm
    masked_map_E = hp.alm2map(alm_E,nside=2048)*masksymm
    alm_masked_T = hp.map2alm(masked_map_T,lmax=2048)
    alm_masked_E = hp.map2alm(masked_map_E,lmax=2048)
    hp.write_alm('smica_symmmasked_T_noise_alm_'+str(i)+'.fits',alm_masked_T)
    hp.write_alm('smica_symmmasked_E_noise_alm_'+str(i)+'.fits',alm_masked_E)

## $N_{\ell}$

In [ ]:
c_nthreads_limit(12)

nsim = 300

cl_nilc_T_mean = np.zeros(2048+1)
cl_nilc_E_mean = np.zeros(2048+1)
cl_smica_T_mean = np.zeros(2048+1)
cl_smica_E_mean = np.zeros(2048+1)

for i in tqdm(range(nsim)):
    almT = hp.read_alm('nilc_symmmasked_T_noise_alm_'+str(i)+'.fits')
    almE = hp.read_alm('/nilc_symmmasked_E_noise_alm_'+str(i)+'.fits')
    clT = hp.alm2cl(almT)
    clE = hp.alm2cl(almE)
    cl_nilc_T_mean = cl_nilc_T_mean + clT/nsim
    cl_nilc_E_mean = cl_nilc_E_mean + clE/nsim

hp.write_cl('nl_nilc_T_symmmasked.fits',cl_nilc_T_mean)
hp.write_cl('nl_nilc_E_symmmasked.fits',cl_nilc_E_mean)
    
for i in tqdm(range(nsim)):
    almT = hp.read_alm('smica_symmmasked_T_noise_alm_'+str(i)+'.fits')
    almE =hp.read_alm('smica_symmmasked_E_noise_alm_'+str(i)+'.fits')
    clT = hp.alm2cl(almT)
    clE = hp.alm2cl(almE)
    cl_smica_T_mean = cl_smica_T_mean + clT/nsim
    cl_smica_E_mean = cl_smica_E_mean + clE/nsim
    
hp.write_cl('nl_smica_T_symmmasked.fits',cl_smica_T_mean)
hp.write_cl('nl_smica_E_symmmasked.fits',cl_smica_E_mean)

## de-Doplerred Noise

### TT

In [ ]:
dd_files_dir = 'dipole_doppler_maps_TT/'

## Applying NILC noise de-Doppler

nsim = 300
lmax_var = 2048
alm = hp.read_alm('nilc_symmmasked_T_noise_alm_'+str(1)+'.fits')

#caching DD data
dipole_doppler_load_low_res(alm=alm,pipeline='nilc',almfile=False,lmax=2048,overbin=10,threads=16,dd_dir=dd_files_dir,verbose=True)
#overbin is leakage removal feature on the edge of the bins (the default bin size is delta_l=200). Let overbin=10 for optimal results.

#Here we set 4 threads, bun below we ser 3 processes -> total of 4x3=12 threads
c_nthreads_limit(4)

for i in range(nsim):
    start = time.time()
    print(i)
    alm = hp.read_alm('debeamed_nilc_T_noise_'+str(i)+'.fits')
    nilc_dd_alm = dipole_doppler_low_res_removal(alm,threads=4,processes=3)
    hp.write_alm('nilc_debeamed_dedopplered_noise_T_sim_'+str(i)+'.fits',nilc_dd_alm)
    end = time.time()
    print(end-start)

## Applying SMICA noise de-Doppler

nsim = 300
lmax_var = 2048
alm = hp.read_alm('smica_symmmasked_T_noise_alm_'+str(1)+'.fits')

#caching DD data
dipole_doppler_load_low_res(alm=alm,pipeline='smica',almfile=False,lmax=2048,overbin=10,threads=16,dd_dir=dd_files_dir,verbose=True)
#overbin is leakage removal feature on the edge of the bins (the default bin size is delta_l=200). Let overbin=10 for optimal results.

#Here we set 4 threads, bun below we ser 3 processes -> total of 4x3=12 threads
c_nthreads_limit(4)

for i in range(nsim):
    start = time.time()
    print(i)
    alm = hp.read_alm('debeamed_smica_T_noise_'+str(i)+'.fits')
    smica_dd_alm = dipole_doppler_low_res_removal(alm,threads=4,processes=3)
    hp.write_alm('smica_debeamed_dedopplered_noise_T_sim_'+str(i)+'.fits',smica_dd_alm)
    end = time.time()
    print(end-start)


In [ ]:
#Maybe you will have to kill and restart the kernel before proceed.

### EE

In [ ]:
dd_files_dir = 'dipole_doppler_maps_EE/'

## Applying NILC noise de-Doppler

nsim = 300
lmax_var = 2048
alm = hp.read_alm('nilc_symmmasked_E_noise_alm_'+str(1)+'.fits')

#caching DD data
dipole_doppler_load_low_res_pol(alm=alm,pipeline='nilc',almfile=False,lmax=2048,overbin=10,threads=16,dd_dir=dd_files_dir,verbose=True)
#overbin is leakage removal feature on the edge of the bins (the default bin size is delta_l=200). Let overbin=10 for optimal results.

#Here we set 4 threads, bun below we ser 3 processes -> total of 4x3=12 threads
c_nthreads_limit(4)

for i in range(nsim):
    start = time.time()
    print(i)
    alm = hp.read_alm('debeamed_nilc_E_noise_'+str(i)+'.fits')
    nilc_dd_alm = dipole_doppler_low_res_removal_pol(alm,threads=4,processes=3)
    hp.write_alm('nilc_debeamed_dedopplered_noise_E_sim_'+str(i)+'.fits',nilc_dd_alm)
    end = time.time()
    print(end-start)

## Applying SMICA noise de-Doppler

nsim = 300
lmax_var = 2048
alm = hp.read_alm('smica_symmmasked_E_noise_alm_'+str(1)+'.fits')

#caching DD data
dipole_doppler_load_low_res_pol(alm=alm,pipeline='smica',almfile=False,lmax=2048,overbin=10,threads=16,dd_dir=dd_files_dir,verbose=True)
#overbin is leakage removal feature on the edge of the bins (the default bin size is delta_l=200). Let overbin=10 for optimal results.

#Here we set 4 threads, bun below we ser 3 processes -> total of 4x3=12 threads
c_nthreads_limit(4)

for i in range(nsim):
    start = time.time()
    print(i)
    alm = hp.read_alm('debeamed_smica_E_noise_'+str(i)+'.fits')
    smica_dd_alm = dipole_doppler_low_res_removal_pol(alm,threads=4,processes=3)
    hp.write_alm('smica_debeamed_dedopplered_noise_E_sim_'+str(i)+'.fits',smica_dd_alm)
    end = time.time()
    print(end-start)
